In [1]:
import pandas as pd
import numpy as np
import re

# 파일 읽어오기
df = pd.read_csv('DE_JD.csv')
df.head()

,채용제목,회사명,근무지역,경력 요구사항,고용조건,주요업무,자격요건,우대사항,링크
0,[Gopax]Data Engineer,스트리미(Streami),서울 강남구,경력 2-6년,NaN,"• 스트리미 시스템의 데이터관리, 추출 및 분석\n• Data Warehouse /...","• 전산학과, 수학, 통계 혹은 관련 분야의 학위 혹은 동등한 조건\r\n• SQL...",• Elastic Stack 사용 경험\r\n• AWS Aurora 혹은 Athen...,https://www.wanted.co.kr/wd/306548
1,데이터 시스템 엔지니어,누아,서울 중구,신입-경력 2년,3개월 수습 후 정규직,"• 항공·여행 산업에서 발생하는 비정형 텍스트 데이터(공지사항, 이메일, 채팅 등)...","• 주어진 문제를 논리적으로 분석하고, 현실적인 해결 방향을 스스로 도출할 수 있는...",• 항공·여행 산업에 대한 관심 또는 관련 경험\r\n• 자료구조의 내부 동작 원리...,https://www.wanted.co.kr/wd/287060
2,데이터 엔지니어 (Google Analytics),이노션,서울 강남구,경력 1년 이상,계약직,• Web/App 내 사용자 행동 데이터 수집 환경 구축 및 운영 (Google A...,• 학사 이상\r\n• 총 경력 1년차 이상 필수\r\n• Web/App 내 Goo...,"• 개발 언어 활용 역량\r\n• Javascript, HTML, CSS\r\n• ...",https://www.wanted.co.kr/wd/305871
3,Data Engineer : 상품 카탈로그,라포랩스,서울 강남구,경력 1년 이상,NaN,"• 상품 카탈로그 시스템의 데이터 파이프라인을 설계, 구축, 운영합니다.\r\n• ...",• 데이터 엔지니어링 경력이 1-3년 또는 이에 준하는 경험·역량을 보유하신 분\r...,"• 커머스, 이커머스 도메인에서 데이터 엔지니어링 경험을 보유하신 분\r\n• 상품...",https://www.wanted.co.kr/wd/305808
4,[데이터플랫폼팀] 데이터 엔지니어,카페24,서울 동작구,경력 2-10년,NaN,• 데이터 인프라 아키텍처 설계 및 운영 자동화\r\nㄴ 대규모 BigData Cl...,"• 2 ~ 10년 경력\r\n• 데이터 플랫폼, 데이터 엔지니어링 또는 관련 분야 ...",• ClickHouse 또는 대규모 OLAP DB 성능 최적화 및 운영 경험\r\n...,https://www.wanted.co.kr/wd/305759


In [2]:
# 결측치 확인
count_title_null = df['채용제목'].isnull().sum()
count_company_name_null = df['회사명'].isnull().sum()
count_work_area_null = df['근무지역'].isnull().sum()
count_requirements_null = df['경력 요구사항'].isnull().sum()
count_conditions_null = df['고용조건'].isnull().sum()
count_tasks_null = df['주요업무'].isnull().sum()
count_qualification_null = df['자격요건'].isnull().sum()
count_preferential_null = df['우대사항'].isnull().sum()
count_link_null = df['링크'].isnull().sum()

print(f"채용제목 결측치 수: {count_title_null}")
print(f"회사명 결측치 수: {count_company_name_null}")
print(f"근무지역 결측치 수: {count_work_area_null}")
print(f"경력 요구사항 결측치 수: {count_requirements_null}")
print(f"고용조건 결측치 수: {count_conditions_null}")
print(f"주요업무 결측치 수: {count_tasks_null}")
print(f"자격요건 결측치 수: {count_qualification_null}")
print(f"우대사항 결측치 수: {count_preferential_null}")
print(f"링크 결측치 수: {count_link_null}")

채용제목 결측치 수: 0
회사명 결측치 수: 0
근무지역 결측치 수: 0
경력 요구사항 결측치 수: 0
고용조건 결측치 수: 67
주요업무 결측치 수: 0
자격요건 결측치 수: 0
우대사항 결측치 수: 0
링크 결측치 수: 0


In [ ]:
# 결측치 대체
df_col_replace = df.fillna({'고용조건': '미기재'})
df_col_replace.head()

In [ ]:
# 중복 값 없애기
df_col_replace.drop_duplicates(subset=['채용제목', '회사명', '링크'])

In [4]:
# 경력요구사항 컬럼명
exp_col = '경력 요구사항'

def parse_experience(exp_str):
    # 기본값 설정
    min_exp = np.nan
    max_exp = np.nan
    is_entry = False

    if pd.isna(exp_str):
        return pd.Series([min_exp, max_exp, is_entry])

    exp_str = exp_str.strip()

    # 신입 포함 여부 체크
    if '신입' in exp_str:
        is_entry = True

    # 형태별 분리 처리
    # 1) 신입-경력 2년
    match = re.match(r'신입-경력\s*(\d+)년', exp_str)
    if match:
        min_exp = 0
        max_exp = int(match.group(1))
        return pd.Series([min_exp, max_exp, is_entry])

    # 2) 경력 2-6년
    match = re.match(r'경력\s*(\d+)[\-~](\d+)년', exp_str)
    if match:
        min_exp = int(match.group(1))
        max_exp = int(match.group(2))
        return pd.Series([min_exp, max_exp, is_entry])

    # 3) 경력 3년 이상
    match = re.match(r'경력\s*(\d+)년 이상', exp_str)
    if match:
        min_exp = int(match.group(1))
        max_exp = np.nan  # 상한 없음
        return pd.Series([min_exp, max_exp, is_entry])

    # 4) 신입 / 신입∙인턴 / 인턴
    if exp_str == '신입' or exp_str == '신입∙인턴' or exp_str == '인턴':
        min_exp = 0
        max_exp = 0
        return pd.Series([min_exp, max_exp, is_entry])

    # 기타 문구는 결측 처리
    return pd.Series([np.nan, np.nan, is_entry])

# 파싱 결과를 새로운 컬럼으로 추가
df_col_replace[['경력_최소', '경력_최대', '신입가능']] = df_col_replace[exp_col].apply(parse_experience)

# 결과 확인
print(df_col_replace[[exp_col, '경력_최소', '경력_최대', '신입가능']].head(10))

       경력 요구사항  경력_최소  경력_최대   신입가능
0      경력 2-6년      2    6.0  False
1     신입-경력 2년      0    2.0   True
2     경력 1년 이상      1    NaN  False
3     경력 1년 이상      1    NaN  False
4     경력 2-10년      2   10.0  False
5           신입      0    0.0   True
6     경력 3년 이상      3    NaN  False
7      경력 2-6년      2    6.0  False
8  신입-경력 10년\r      0   10.0   True
9      경력 2-7년      2    7.0  False


In [5]:
# 근무지역 컬럼명
loc_col = '근무지역'

# 시도, 구군 분리 함수
def split_region(region):
    if pd.isna(region):
        return pd.Series([None, None])
    parts = region.split()
    sido = parts[0] if len(parts) > 0 else None
    gugun = ' '.join(parts[1:]) if len(parts) > 1 else None
    return pd.Series([sido, gugun])

# 근무지역에서 시도, 구군 분리하여 컬럼 추가
df_col_replace[['시도', '구군']] = df_col_replace[loc_col].apply(split_region)

# 결과 확인
print(df_col_replace[[loc_col, '시도', '구군']].head(10))

     근무지역  시도   구군
0  서울 강남구  서울  강남구
1   서울 중구  서울   중구
2  서울 강남구  서울  강남구
3  서울 강남구  서울  강남구
4  서울 동작구  서울  동작구
5  경북 포항시  경북  포항시
6  서울 관악구  서울  관악구
7  서울 강남구  서울  강남구
8  서울 서초구  서울  서초구
9  서울 강남구  서울  강남구
